# Transfer Learning Models

## Libraries to Import

In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from PIL import Image
import keras
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras import models, layers, optimizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization

import os
import gc
from timeit import default_timer as timer

## Define Image Directories

In [2]:
train_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/training/'
test_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/testing/'
val_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/validation/'

Create flow_from_directory objects for Keras model. Using a downsized dataset with class balance for local model running.

In [3]:
train_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(train_dir,
                                                                  target_size = (256,256), 
                                                                  color_mode='rgb',
                                                                  class_mode='binary')
val_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(val_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')
test_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(test_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')

Found 1346 images belonging to 2 classes.
Found 593 images belonging to 2 classes.
Found 947 images belonging to 2 classes.


## Xception -- 5 epochs

In [19]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

83689472/83683744 [==============================] - 3s 0us/step


In [22]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [23]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=5, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 140s 3s/step - loss: 0.6371 - accuracy: 0.6612 - recall: 0.6657 - val_loss: 0.5866 - val_accuracy: 0.7285 - val_recall: 0.7381
Epoch 2/5
43/43 [==============================] - 157s 4s/step - loss: 0.5689 - accuracy: 0.7348 - recall: 0.7504 - val_loss: 0.5515 - val_accuracy: 0.7369 - val_recall: 0.7517
Epoch 3/5
43/43 [==============================] - 149s 3s/step - loss: 0.5432 - accuracy: 0.7422 - recall: 0.7221 - val_loss: 0.5315 - val_accuracy: 0.7521 - val_recall: 0.7789
Epoch 4/5
43/43 [==============================] - 147s 3s/step - loss: 0.5259 - accuracy: 0.7600 - recall: 0.7340 - val_loss: 0.5498 - val_accuracy: 0.7167 - val_recall: 0.8946
Epoch 5/5
43/43 [==============================] - 145s 3s/step - loss: 0.5078 - accuracy: 0.7615 - recall: 0.7563 - val_loss: 0.5155 - val_accuracy: 0.7589 - val_recall: 0.7891
Total Time Elapsed:  12  minutes  34  seconds


In [24]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 65s 2s/step - loss: 0.9548 - accuracy: 0.4340 - recall: 0.2685


## Xception -- 15 epochs

In [5]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [6]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [7]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=15, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/15
43/43 [==============================] - 142s 3s/step - loss: 0.6473 - accuracy: 0.6367 - recall: 0.6865 - val_loss: 0.6001 - val_accuracy: 0.7403 - val_recall: 0.5782
Epoch 2/15
43/43 [==============================] - 152s 4s/step - loss: 0.5757 - accuracy: 0.7318 - recall: 0.6909 - val_loss: 0.5587 - val_accuracy: 0.7302 - val_recall: 0.7959
Epoch 3/15
43/43 [==============================] - 150s 3s/step - loss: 0.5401 - accuracy: 0.7511 - recall: 0.7251 - val_loss: 0.5347 - val_accuracy: 0.7504 - val_recall: 0.7823
Epoch 4/15
43/43 [==============================] - 148s 3s/step - loss: 0.5240 - accuracy: 0.7541 - recall: 0.7400 - val_loss: 0.5265 - val_accuracy: 0.7521 - val_recall: 0.7891
Epoch 5/15
43/43 [==============================] - 148s 3s/step - loss: 0.5080 - accuracy: 0.7689 - recall: 0.7578 - val_loss: 0.5233 - val_accuracy: 0.7504 - val_recall: 0.8163
Epoch 6/15
43/43 [==============================] - 147s 3s/step - loss: 0.5003 - accuracy: 0.7704 - reca

In [8]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 68s 2s/step - loss: 1.0511 - accuracy: 0.4636 - recall: 0.2778


## VGG16 -- 5 epochs

In [5]:
base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [6]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [7]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=5, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 322s 7s/step - loss: 0.6880 - accuracy: 0.5572 - recall: 0.3314 - val_loss: 0.6756 - val_accuracy: 0.7150 - val_recall: 0.7721
Epoch 2/5
43/43 [==============================] - 319s 7s/step - loss: 0.6673 - accuracy: 0.6709 - recall: 0.6256 - val_loss: 0.6589 - val_accuracy: 0.7184 - val_recall: 0.8333
Epoch 3/5
43/43 [==============================] - 316s 7s/step - loss: 0.6510 - accuracy: 0.7065 - recall: 0.6642 - val_loss: 0.6443 - val_accuracy: 0.7234 - val_recall: 0.7925
Epoch 4/5
43/43 [==============================] - 335s 8s/step - loss: 0.6362 - accuracy: 0.7065 - recall: 0.7073 - val_loss: 0.6331 - val_accuracy: 0.7302 - val_recall: 0.7619
Epoch 5/5
43/43 [==============================] - 312s 7s/step - loss: 0.6255 - accuracy: 0.7147 - recall: 0.7132 - val_loss: 0.6225 - val_accuracy: 0.7319 - val_recall: 0.7619
Total Time Elapsed:  27  minutes  12  seconds


In [8]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 148s 5s/step - loss: 0.7729 - accuracy: 0.4234 - recall: 0.2238


## VGG16 -- 15 epochs

In [9]:
base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [10]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [11]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=15, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/15
43/43 [==============================] - 309s 7s/step - loss: 0.6808 - accuracy: 0.6263 - recall: 0.7831 - val_loss: 0.6708 - val_accuracy: 0.6071 - val_recall: 0.8878
Epoch 2/15
43/43 [==============================] - 312s 7s/step - loss: 0.6611 - accuracy: 0.6471 - recall: 0.7385 - val_loss: 0.6565 - val_accuracy: 0.6779 - val_recall: 0.8095
Epoch 3/15
43/43 [==============================] - 313s 7s/step - loss: 0.6473 - accuracy: 0.6857 - recall: 0.7207 - val_loss: 0.6440 - val_accuracy: 0.7133 - val_recall: 0.7483
Epoch 4/15
43/43 [==============================] - 314s 7s/step - loss: 0.6364 - accuracy: 0.7058 - recall: 0.6374 - val_loss: 0.6365 - val_accuracy: 0.6813 - val_recall: 0.8197
Epoch 5/15
43/43 [==============================] - 312s 7s/step - loss: 0.6262 - accuracy: 0.7036 - recall: 0.7207 - val_loss: 0.6291 - val_accuracy: 0.6728 - val_recall: 0.8265
Epoch 6/15
43/43 [==============================] - 319s 7s/step - loss: 0.6147 - accuracy: 0.7043 - reca

In [12]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 151s 5s/step - loss: 0.8086 - accuracy: 0.4615 - recall: 0.3395


## InceptionResNetV2 -- 5 epochs

In [4]:
base_model = keras.applications.InceptionResNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

219062272/219055592 [==============================] - 8s 0us/step


In [5]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [6]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=5, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 156s 4s/step - loss: 0.6612 - accuracy: 0.6040 - recall: 0.6270 - val_loss: 0.6085 - val_accuracy: 0.6155 - val_recall: 0.9184
Epoch 2/5
43/43 [==============================] - 155s 4s/step - loss: 0.6047 - accuracy: 0.6932 - recall: 0.6909 - val_loss: 0.5432 - val_accuracy: 0.7774 - val_recall: 0.8027
Epoch 3/5
43/43 [==============================] - 154s 4s/step - loss: 0.5666 - accuracy: 0.7266 - recall: 0.7192 - val_loss: 0.5370 - val_accuracy: 0.8061 - val_recall: 0.6837
Epoch 4/5
43/43 [==============================] - 154s 4s/step - loss: 0.5538 - accuracy: 0.7221 - recall: 0.6954 - val_loss: 0.5089 - val_accuracy: 0.7555 - val_recall: 0.8571
Epoch 5/5
43/43 [==============================] - 157s 4s/step - loss: 0.5404 - accuracy: 0.7340 - recall: 0.7251 - val_loss: 0.4899 - val_accuracy: 0.8128 - val_recall: 0.8027
Total Time Elapsed:  13  minutes  17  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 74s 2s/step - loss: 0.8767 - accuracy: 0.4657 - recall: 0.2963


## InceptionResNetV2 -- 15 epochs

In [8]:
base_model = keras.applications.InceptionResNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [9]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [10]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=15, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/15
43/43 [==============================] - 163s 4s/step - loss: 0.6693 - accuracy: 0.6129 - recall: 0.5795 - val_loss: 0.6048 - val_accuracy: 0.6374 - val_recall: 0.9252
Epoch 2/15
43/43 [==============================] - 158s 4s/step - loss: 0.5991 - accuracy: 0.7013 - recall: 0.6969 - val_loss: 0.5703 - val_accuracy: 0.7437 - val_recall: 0.5408
Epoch 3/15
43/43 [==============================] - 158s 4s/step - loss: 0.5762 - accuracy: 0.6961 - recall: 0.6999 - val_loss: 0.5371 - val_accuracy: 0.7285 - val_recall: 0.9048
Epoch 4/15
43/43 [==============================] - 159s 4s/step - loss: 0.5621 - accuracy: 0.7296 - recall: 0.7251 - val_loss: 0.5015 - val_accuracy: 0.8061 - val_recall: 0.7823
Epoch 5/15
43/43 [==============================] - 158s 4s/step - loss: 0.5442 - accuracy: 0.7325 - recall: 0.7325 - val_loss: 0.4957 - val_accuracy: 0.8094 - val_recall: 0.7619
Epoch 6/15
43/43 [==============================] - 159s 4s/step - loss: 0.5264 - accuracy: 0.7370 - reca

In [11]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 74s 2s/step - loss: 0.8598 - accuracy: 0.4879 - recall: 0.3534
